# Day 9: Smoke Basin

In [1]:
example = """2199943210
3987894921
9856789892
8767896789
9899965678"""

In [2]:
def parse(input):
    """Parses heightmap from input."""
    return [[int(digit) for digit in row] for row in input.split()]

parse(example)

[[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
 [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
 [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
 [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
 [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]]

In [3]:
import math

def adjacencies(heightmap):
    """Returns map of adjacent elements for each element in heightmap."""
    return [
        [(
            # Up.
            heightmap[i - 1][j] if i > 0 else math.inf,
            # Down. 
            heightmap[i + 1][j] if i + 1 < len(heightmap) else math.inf,
            # Left.
            heightmap[i][j - 1] if j > 0 else math.inf,
            # Right.
            heightmap[i][j + 1] if j + 1 < len(row) else math.inf
        ) for j, _ in enumerate(row)] 
        for i, row in enumerate(heightmap)
    ]

adjacencies(parse(example))

[[(inf, 3, inf, 1),
  (inf, 9, 2, 9),
  (inf, 8, 1, 9),
  (inf, 7, 9, 9),
  (inf, 8, 9, 4),
  (inf, 9, 9, 3),
  (inf, 4, 4, 2),
  (inf, 9, 3, 1),
  (inf, 2, 2, 0),
  (inf, 1, 1, inf)],
 [(2, 9, inf, 9),
  (1, 8, 3, 8),
  (9, 5, 9, 7),
  (9, 6, 8, 8),
  (9, 7, 7, 9),
  (4, 8, 8, 4),
  (3, 9, 9, 9),
  (2, 8, 4, 2),
  (1, 9, 9, 1),
  (0, 2, 2, inf)],
 [(3, 8, inf, 8),
  (9, 7, 9, 5),
  (8, 6, 8, 6),
  (7, 7, 5, 7),
  (8, 8, 6, 8),
  (9, 9, 7, 9),
  (4, 6, 8, 8),
  (9, 7, 9, 9),
  (2, 8, 8, 2),
  (1, 9, 9, inf)],
 [(9, 9, inf, 7),
  (8, 8, 8, 6),
  (5, 9, 7, 7),
  (6, 9, 6, 8),
  (7, 9, 7, 9),
  (8, 6, 8, 6),
  (9, 5, 9, 7),
  (8, 6, 6, 8),
  (9, 7, 7, 9),
  (2, 8, 8, inf)],
 [(8, inf, inf, 8),
  (7, inf, 9, 9),
  (6, inf, 8, 9),
  (7, inf, 9, 9),
  (8, inf, 9, 6),
  (9, inf, 9, 5),
  (6, inf, 6, 6),
  (7, inf, 5, 7),
  (8, inf, 6, 8),
  (9, inf, 7, inf)]]

Low points match example.

In [4]:
def low_points(heightmap):
    """Returns True for low points in heightmap."""
    return [
        [
            # Is height less than all adjacent heights?
            all(height < adjacent for adjacent in adjacent_heights) 
            for height, adjacent_heights in zip(heights, adjacency_map)
        ] for heights, adjacency_map in zip(heightmap, adjacencies(heightmap))
    ]

    return list(zip(heightmap, adjacencies(heightmap)))

low_points(parse(example))

[[False, True, False, False, False, False, False, False, False, True],
 [False, False, False, False, False, False, False, False, False, False],
 [False, False, True, False, False, False, False, False, False, False],
 [False, False, False, False, False, False, False, False, False, False],
 [False, False, False, False, False, False, True, False, False, False]]

Sum of low point risks matches example.

In [5]:
def sum_low_point_risk(heightmap):
    """Sums low point risks in heightmap."""
    return sum(
        # Risk is 1 its height.
        1 + height
        for heights_row, low_points_row in zip(heightmap, low_points(heightmap))
        for height, low_point in zip(heights_row, low_points_row) if low_point
    )
    
sum_low_point_risk(parse(example))

15

Sum of low point risks in input.

In [6]:
sum_low_point_risk(parse(open('day-9-input.txt').read()))

603

# Part two

In [7]:
def basins(heightmap):
    """Finds basins in heightmap."""
    adjacent = adjacencies(heightmap)
    
    # Iterate over low points. 
    for i, j in {
        (i, j)
        for i, row in enumerate(low_points(heightmap))
        for j, low_point in enumerate(row) if low_point
    }:
        # Basins are seeded with low points. 
        basin = {(i, j)}
        
        while True:
            candidates = set()

            for i, j in basin:
                adjacent_coords = ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1))

                # Iterate over adjacent coordinates
                for m, n in ((i, j) for i, j in adjacent_coords if i > -1 and i < len(heightmap) and j > -1 and j < len(heightmap[0])):
                    try:
                        # It's a candidate if m,n flows into i,j.
                        if heightmap[m][n] < 9 and heightmap[m][n] > heightmap[i][j]:
                            candidates.add((m, n))
                    except IndexError:
                        pass
                
            # Terminate if candidates are contained in basin.
            if candidates <= basin:
                break
            else:
                basin |= candidates

        yield basin

# list(basins(parse(example)))

basin_points = {point for basin in basins(parse(example)) for point in basin}
basin_points
[
    ['o' if (i, j) in basin_points else '.' for j, _ in enumerate(row)]
    for i, row in enumerate(parse(example)) 
]

[['o', 'o', '.', '.', '.', 'o', 'o', 'o', 'o', 'o'],
 ['o', '.', 'o', 'o', 'o', '.', 'o', '.', 'o', 'o'],
 ['.', 'o', 'o', 'o', 'o', 'o', '.', 'o', '.', 'o'],
 ['o', 'o', 'o', 'o', 'o', '.', 'o', 'o', 'o', '.'],
 ['.', 'o', '.', '.', '.', 'o', 'o', 'o', 'o', 'o']]

Sizes of basins.

In [8]:
[len(basin) for basin in basins(parse(example))]

[3, 9, 9, 14]

Three largest.

In [9]:
sorted([len(basin) for basin in basins(parse(example))][-3:])

[9, 9, 14]

Product of the largest three.

In [10]:
math.prod(sorted([len(basin) for basin in basins(parse(example))][-3:]))

1134

Product of the largest three basins in input.

In [11]:
basins

<function __main__.basins(heightmap)>

In [12]:
math.prod(sorted([len(basin) for basin in basins(parse(open('day-9-input.txt').read()))])[-3:])

786780